In [ ]:
#instalação das libs
!pip install selenium pandas

In [ ]:
#Importação das libs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
#Inicializando a instância do Google Chrome
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
#Define o url
url = 'https://jurisprudencia.tst.jus.br'
driver.get(url)

In [ ]:
#Insere o termo a ser pesquisado no campo 'Contendo as palavras (e)' E clica ENTER para pesquisar
termo_pesquisa = input('Digite o termo a ser pesquisado: ')
campo_pesquisa = driver.find_element(By.XPATH, '//*[@id="campoTxtOperadorE"]')
campo_pesquisa.send_keys(termo_pesquisa)
campo_pesquisa.send_keys(Keys.ENTER)

In [ ]:
#Altera a visualização para 100 resultados por página
dropdown_button = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="main"]/header/div[2]/div[1]/div[3]/div/div/div[2]/div/div')))
    
dropdown_button.click()

# Espera que a opção 100 esteja presente e visível
option_to_select = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="menu-"]/div[2]/ul/li[5]'))
)

# Clica na opção desejada
option_to_select.click()

In [ ]:
#Extraindo os campos necessarios

In [ ]:
#Extrai numeros dos processos
lista_processos = []

processos = driver.find_elements(By.XPATH, '//a[contains(@href, "#")]')

for processo in processos:
    text = processo.text
    # Adiciona o texto à lista apenas se não for igual aos termos especificados
    if text not in ['Ementa para citação', 'Dispositivo','Justificativa de Voto Vencido', 'Voto Convergente']:
        lista_processos.append(text)
        
len(lista_processos)

In [ ]:
#Extrai as turmas que julgaram o documento
lista_turmas = []

# Localiza todos os elementos <div> que contenham o texto "Orgão Judicante:"
turmas = driver.find_elements(By.XPATH, '//div[contains(text(), "Orgão Judicante:")]')

# Itera sobre os elementos encontrados e extrai o texto após "Orgão Judicante:"
for turma in turmas:
    # Obtém o texto do elemento
    text = turma.text
    # Extrai o texto após "Orgão Judicante:" e apenas na lista de turmas
    orgao_judicante = text.split('Orgão Judicante:')[1].strip()
    lista_turmas.append(orgao_judicante)
    # Exibe o resultado

len(lista_turmas)

In [ ]:
#Extrai os nomes do relatores dos documentos
lista_relator = []

# Localiza todos os elementos <div> que contenham o texto "Relator"
relatores = driver.find_elements(By.XPATH, '//div[contains(text(), "Relator")]')

# Itera sobre os elementos encontrados e extrai o texto após ":"
for relator in relatores:
    # Obtém o texto do elemento
    text = relator.text
    # Extrai o texto após ":" e apensa na lista de relatores
    relator_ext = text.split(':')[1].strip()
    lista_relator.append(relator_ext)

len(lista_relator)

In [ ]:
#Extrai as datas de publicação dos documentos
lista_datapub = []

# Localiza todos os elementos <div> que contenham o texto "Publicação"
dataspub = driver.find_elements(By.XPATH, '//div[contains(text(), "Publicação")]')

# Itera sobre os elementos encontrados e extrai o texto após ":"
for data in dataspub:
    # Obtém o texto do elemento
    text = data.text
    # Extrai o texto após ":" e apensa na lista de datas de publicação
    datapub_ext = text.split(':')[1].strip()
    lista_datapub.append(datapub_ext)
    # Exibe o resultado

len(lista_datapub)

In [ ]:
#Extrai o tipo de documento 
lista_tipos = []

# Localiza todos os elementos <div> que contenham o texto "Tipo de Documento:"
tipos = driver.find_elements(By.XPATH, '//div[contains(text(), "Tipo de Documento: ")]')

# Itera sobre os elementos encontrados e extrai o texto após ":"
for tipo in tipos:
    # Obtém o texto do elemento
    text = tipo.text
    # Extrai o texto após ":" e apensa na lista de tipos
    tipo_ext = text.split(':')[1].strip()
    lista_tipos.append(tipo_ext)
    # Exibe o resultado

len(lista_tipos)

In [ ]:
#Extrai as ementas dos documentos
lista_ementas = []

# Localiza todos os elementos CLASS_NAME iguais a "autoHeight"
ementas = driver.find_elements(By.CLASS_NAME, 'autoHeight')

# Itera sobre os elementos encontrados
for ementa in ementas:
    # Obtém o texto do elemento e apensa na lista de ementas
    text = ementa.text
    lista_ementas.append(text)

del lista_ementas[1] #deleta o elemento que foi identificado como estranho
len(lista_ementas)

In [ ]:
#Extrai o inteiro teor dos documentos encontrados abertos em nova guia
lista_teor = []

# Localiza todos os elementos que abrem em nova guia
click_teor = driver.find_elements(By.CLASS_NAME, 'open_in_new_window')

# Itera sobre os elementos encontrados e clica neles para então extrair o texto da nova guia
for teor in click_teor:
    # Abre o link em uma nova guia
    ActionChains(driver).key_down(Keys.CONTROL).click(teor).key_up(Keys.CONTROL).perform()

    # Muda para a nova guia aberta
    driver.switch_to.window(driver.window_handles[-1])

    # Aguarda a nova página carregar completamente
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

    # Extrai o texto da nova página
    text = driver.find_element(By.TAG_NAME, 'html').text
    lista_teor.append(text)

    # Fecha a nova guia e retorna para a guia original
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

len (lista_teor)

In [ ]:
#Fecha o navegador 
driver.quit()

In [ ]:
#Converte os dados extraídos em DataFrame
dictDF = {'Processo':lista_processos,
          'Turma': lista_turmas,
         'Relator':lista_relator,
         'Data de Publicação': lista_datapub,
         'Tipo de Decisão': lista_tipos,
         'Ementa':lista_ementas,
         'Inteiro Teor':lista_teor}
df = pd.DataFrame(dictDF)

In [ ]:
#Salva o data frame em um arquivo excel
nome_do_arquivo = input('Como deseja nomear o arquivo? ')
df.to_excel(nome_do_arquivo)